In [1]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")
default_model = os.getenv("OPENAI_DEFAULT_MODEL")
redis_host = os.getenv("REDIS_HOST")
redis_password = os.getenv("REDIS_PASSWORD")
redis_port = os.getenv("REDIS_PORT")
tavily_key = os.getenv("TAVILY_API_KEY")


In [2]:
from typing import Annotated,List
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict
from typing import Annotated, List, Literal
from dotenv import load_dotenv
import os
from langgraph.types import interrupt, Command
from operator import add

c:\Users\edukd\miniconda3\envs\myKDA2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
from langchain_core.messages import ToolMessage
import json
class ToolNode:
    def __init__(self, tools):
        self.tools_by_name = {tool.name: tool for tool in tools} #툴 정보매핑
    def __call__(self, inputs):
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No messages found in inputs.")
        outputs = []
        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call['name']].invoke(tool_call['args'])
            tool_message = ToolMessage(
                content=json.dumps(tool_result),
                name=tool_call['name'],
                tool_call_id=tool_call['id'],
            )
            outputs.append(tool_message)
        return {"messages": outputs}

In [22]:
from langchain_core.tools import tool
@tool(description="주어진 지역의 현재 날씨를 반환")
def get_weather(location:str) -> str:
    if location.lower() in ['서울', '부산']:
        return f"{location}의 현재 날씨는 23도입니다."
    else:
        return f"{location}의 날씨 정보가 없습니다."
@tool(description="가장 시원한 도시들의 목록을 반환")
def get_weather1(location:str) -> str:
    return "서울, 부산"

In [23]:
tools = [get_weather, get_weather1]
tool_node = ToolNode(tools=tools)

In [24]:
class State(TypedDict):
    messages : Annotated[list, add_messages]

In [25]:
llm = ChatOpenAI(model_name = default_model)

In [26]:
def chatbot(state:State):
    response = llm.invoke(state['messages'])
    return {"messages":[response]}

In [27]:
workflow = StateGraph(State)

workflow.add_node("tools", tool_node)

workflow.add_edge(START, "tools")
workflow.add_edge("tools", END)
graph = workflow.compile()

In [30]:
from langchain_core.messages import HumanMessage, AIMessage

tool_call_message = AIMessage(
    content="",
    tool_calls=[
        {
            "name" : "get_weather",
            "args" : {"location":"서울"},
            "id" : "tool_call_1",
            "type" : "tool_call"
        }
    ]
)

state = {"messages":[tool_call_message]}

response = graph.invoke(state)
s = response["messages"][-1].content
print(s.encode("utf-8").decode("unicode_escape"))

"서울의 현재 날씨는 23도입니다."
